In [1]:
import os
import angr, claripy
import networkx as nx

filePath = '/home/nahid/reverse/binaries/fauxware'
proj = angr.Project(filePath, load_options={"auto_load_libs": False})

In [5]:


cfg = proj.analyses.CFGEmulated(keep_state=True,
                             state_add_options=angr.sim_options.refs,
                             context_sensitivity_level=2)



WARNING  | 2023-06-09 15:37:50,514 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2023-06-09 15:37:50,515 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2023-06-09 15:37:50,515 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2023-06-09 15:37:50,516 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2023-06-09 15:37:50,516 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2023-06-09 15:37:50,517 | angr.storage.memory_mixins.default_filler_mixin | Filling register 

In [6]:
# Generate the control dependence graph
cdg = proj.analyses.CDG(cfg)

# Build the data dependence graph. It might take a while. Be patient!
ddg = proj.analyses.DDG(cfg)

In [7]:
digraph = ddg.graph
digraph = digraph.to_undirected()

In [8]:
connected_components = nx.connected_components(digraph)

In [9]:
for compo in connected_components:
    comp_addresses = []
    for node in compo:
        print(node , hex(node.ins_addr))
#         if node[0] not in comp_addresses:
#             comp_addresses.append(node[0])
#         # print( node , file = log)


#     comp_addresses = [ ( int(i, 16)) for i in comp_addresses]
#     comp_addresses.sort()
#     comp_addresses= [ hex(i) for i in comp_addresses]
#     print("\n\n" , file = log)
#     for ca in comp_addresses:
#         print(ca, file = log)


<0x400582 id=0x400580[4] contextless> 0x400582
<0x400582 id=0x400580[3] contextless> 0x400582
<0x4008b5 id=0x4008af[22] contextless> 0x4008b5
<0x4008b3 id=0x4008af[11] contextless> 0x4008b3
<0x400864 id=0x400846[35] contextless> 0x400864
<0x400828 id=0x400825[7] contextless> 0x400828
<0x4004f7 id=0x4004f3[-2] contextless> 0x4004f7
<0x400850 id=0x400846[12] contextless> 0x400850
<0x400785 id=0x400774[16] contextless> 0x400785
<0x400670 id=0x400664[21] contextless> 0x400670
<0x4008a4 id=0x4008a0[10] contextless> 0x4008a4
<0x4007a0 id=0x4007a0[1] contextless> 0x4007a0
<0x400660 id=0x400660[5] contextless> 0x400660
<0x40079b id=0x40078a[13] contextless> 0x40079b
<0x4007fd id=0x4007e0[22] contextless> 0x4007fd
<0x400894 id=0x400880[27] contextless> 0x400894
<0x400820 id=0x4007e0[55] contextless> 0x400820
<0x4005c2 id=0x4005be[10] contextless> 0x4005c2
<0x400807 id=0x4007e0[31] contextless> 0x400807
<0x400660 id=0x400660[4] contextless> 0x400660
<0x40072c id=0x40071d[27] contextless> 0x40072

In [10]:
with open('angr.s', 'w') as angr_file:
    for func_node in cfg.functions.values():
            for block in func_node.blocks:
                c = block.capstone

                for i in c.insns:
    #                 print(i)
                    print("0x%x:\t%s\t%s" %(i.address, 
                                    i.mnemonic, i.op_str))
                    angr_file.write("0x%x:\t%s\t%s" %(i.address, 
                                    i.mnemonic, i.op_str)+'\n')

0x4004e0:	sub	rsp, 8
0x4004e4:	call	0x4005ac
0x4004e9:	call	0x400640
0x4004ee:	call	0x400880
0x4004f3:	add	rsp, 8
0x4004f7:	ret	
0x400510:	jmp	qword ptr [rip + 0x200aea]
0x400520:	jmp	qword ptr [rip + 0x200ae2]
0x400530:	jmp	qword ptr [rip + 0x200ada]
0x400540:	jmp	qword ptr [rip + 0x200ad2]
0x400550:	jmp	qword ptr [rip + 0x200aca]
0x400560:	jmp	qword ptr [rip + 0x200ac2]
0x400570:	jmp	qword ptr [rip + 0x200aba]
0x400580:	xor	ebp, ebp
0x400582:	mov	r9, rdx
0x400585:	pop	rsi
0x400586:	mov	rdx, rsp
0x400589:	and	rsp, 0xfffffffffffffff0
0x40058d:	push	rax
0x40058e:	push	rsp
0x40058f:	mov	r8, 0x400870
0x400596:	mov	rcx, 0x4007e0
0x40059d:	mov	rdi, 0x40071d
0x4005a4:	call	0x400540
0x4005ac:	sub	rsp, 8
0x4005b0:	mov	rax, qword ptr [rip + 0x200a29]
0x4005b7:	test	rax, rax
0x4005ba:	je	0x4005be
0x4005be:	add	rsp, 8
0x4005c2:	ret	
0x4005bc:	call	rax
0x400640:	cmp	qword ptr [rip + 0x200800], 0
0x400648:	push	rbp
0x400649:	mov	rbp, rsp
0x40064c:	je	0x400660
0x400660:	pop	rbp
0x400661:	ret	
0x4006

In [ ]:



# See where we wanna go... let's go to the exit() call, which is modeled as a
# SimProcedure.
target_func = cfg.kb.functions.function(name="exit")
print(target_func)
# We need the CFGNode instance
target_node = cfg.get_any_node(target_func.addr)

In [ ]:
print(target_node)

In [ ]:
# Let's get a BackwardSlice out of them!
# ``targets`` is a list of objects, where each one is either a CodeLocation
# object, or a tuple of CFGNode instance and a statement ID. Setting statement
# ID to -1 means the very beginning of that CFGNode. A SimProcedure does not
# have any statement, so you should always specify -1 for it.
print('Target Node: ', target_node)
bs = proj.analyses.BackwardSlice(cfg, cdg=cdg, ddg=ddg, targets=[ (target_node, -1) ])



In [ ]:
from angrutils import *
# print(bs.kb.cfgs.cfgs.values())
print(bs.runs_in_slice)
print('cfg_nodes_in_slice',bs.cfg_nodes_in_slice)
print('chosen_statements',bs.chosen_statements)
print('\n\n chosen_exits',bs.chosen_exits)
acfg = bs.annotated_cfg()
print(dir( acfg))


In [ ]:
plot_cfg(cfg, "faux", asminst = True, remove_imports = True, remove_path_terminator=True)  

In [ ]:
bs.dbg_repr()

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt
# # Need to create a layout when doing
# # separate calls to draw nodes and edges
# G= bs.cfg_nodes_in_slice()
# print(G)
# pos = nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
#                        node_color = values, node_size = 500)
# nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos, edgelist=red_edges, edge_color='r', arrows=True)
# nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
# plt.show()